In [23]:
import numpy as np

# switchdoor function takes in the door numbers of the contestant's door choice 
# and the door number with the prize and returns the door number
# that will be offered as a switch
# Scenario 1: The door chosen holds the prize then the host can choose any of the 2 remaining doors for switching
# Scenario 2: The door chosen does not hold the prize then only the door with the prize can be offered as a switch

def switchdoor(choice, prize):   # choice & prize are numpy arrays with elements 1, 2 or 3 representing door numbers
    switch = choice.copy()       # switch will be an array of the same size to contain the results
    
    # scenario 1
    print(choice==prize)
    print(switch[choice==prize])
    switch[choice==prize] = np.mod(choice[choice==prize],3) + 1   # maps 1=>2, 2=>3, 3=>1
    
    # scenario 2
    switch[~(choice==prize)] = prize[~(choice==prize)]            # only choice is the door with the prize 
    
    return switch

# simulation

N = 1000000  # number of simulations

prng = np.random.RandomState()          # pseudo-random number generator for numpy arrays
prize = np.ceil(prng.uniform(0,3,N))    # randomly choose door with prize -> array of size N with elements 1, 2 or 3
choice = np.ceil(prng.uniform(0,3,N))   # randomly select initial choice
switch = switchdoor(choice, prize)      # find the door offered for switch
print(choice)
print(prize)

stay_win = prize == choice      # stay_win = array of true/false elements -> true if contestant chooses stays and wins
stay_win_rate = len(stay_win[stay_win == True]) / N        # win rate i.e win probability if contestant does not switch

switch_win = switch == prize    # switch_win = array of true/false elements -> true if contestant chooses switches and wins
switch_win_rate = len(switch_win[switch_win == True]) / N  # win probability if contestant switches also = 1-stay_win_rate

print('Percentage of games won by staying: {:.1%}'.format(stay_win_rate))
print('Percentage of games won by switching: {:.1%}'.format(switch_win_rate))

[False False  True ...  True  True False]
[3. 2. 2. ... 2. 2. 1.]
[3. 2. 3. ... 2. 1. 2.]
[1. 1. 3. ... 2. 1. 3.]
Percentage of games won by staying: 33.4%
Percentage of games won by switching: 66.6%


In [6]:
%reset -f

import math
import numpy as np

# function that takes the parameters for pricing a option and the number of iterations required in the simulation
# returns the sum of the simulated option prices and the variance of the sample

def MC_BS_EKO_Call(S0,K,B,T,σ,r,it,display=False):
  
    # Step 1: Use random number generators to simulate asset price paths using Risk Neutral probabilities    
    Z = math.sqrt(T)*np.random.standard_normal(it)  # generates normal random variables for the GBM process
    S = S0*np.exp((r-0.5*σ**2)*T + σ*Z)             # simulates an array of final prices of asset at maturity of option

    # Step 2: Compute final value of derivative based on the payoff of the derivative
    KO = S<B                                        # generates an array of True=1 and False=0 based on barrier condition
    V = math.exp(-r*T)*np.maximum(S-K,0)*KO         # generate array of option payoff with asset price & barrier condition
    return S, V

S0 = 120.; K = 100.; B = 120.; T = 1.5; r = 0.02; σ = 0.3    # parameters

N = 10     # number of simulations

S, V = MC_BS_EKO_Call(S0,K,B,T,σ,r,N,True) 

# Step 3: With a suitable number of simulations, compute the discounted expected value through averaging
price = np.average(V)
stderr = np.std(V, ddof=1) / N**0.5   

# ddof = Delta Degrees of Freedom 
# the divisor used in calculations is N - ddof, where N represents the number of elements
# by default ddof is zero.

print('Single engine: total iterations =', N, 'Px={:.4}'.format(price), 'Std err = {:.4}'.format(stderr), '\n')
print(S, '\n')
print(V, '\n')

Single engine: total iterations = 10 Px=0.1661 Std err = 0.1661 

[239.98455905  90.75722478 206.08223524 101.71205052  76.09804941
 126.81191129  76.77791301 138.33233864 167.84900725  86.66266459] 

[0.         0.         0.         1.66145178 0.         0.
 0.         0.         0.         0.        ] 



In [3]:
# First of all, how many cores are available on this PC?
import multiprocessing
print("Number of cores on this PC:", multiprocessing.cpu_count())

Number of cores on this PC: 12


In [9]:
# To get started with ipyparallel:
# 1) At an Anaconda command prompt, enter:
#    conda install ipyparallel or pip install ipyparallel
# 2) Still at the Anaconda command prompt, enter:
#    ipcluster start -n <# of engines for parallel processing>
#    to start the controller and # engines
# 3) Wait until you see this message:
#    "Engines appear to have started successfully"
# 4) Verify everything is working with a Hello, World example:

import ipyparallel as ipp
# The client connects to the cluster of “remote” engines that perfrom the actual computation
# These engines may be on the same machine or on a cluster
rc = ipp.Client() 

# prints the id of the engines
print(rc.ids)

# send the job to all engines
rc[:].apply_sync(lambda : 'Hello world')

# N.B. A really useful reference for learning ipyparallel:
# http://people.duke.edu/~ccc14/sta-663-2016/19C_IPyParallel.html

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


['Hello world',
 'Hello world',
 'Hello world',
 'Hello world',
 'Hello world',
 'Hello world',
 'Hello world',
 'Hello world',
 'Hello world',
 'Hello world']

In [10]:
# A view provides access to a subset of the engines available to the client
# Jobs are submitted to the engines via the view
# A direct view allows the user to explicitly send work specific engines
dview = rc[:]

# map takes a function and iterable and sync blocks the result until all executions are complete
dview.map_sync(lambda x, y, z: x + y + z, range(10), range(10), range(10))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

In [11]:
N = 10000000

%time res = dview.map_sync(lambda x, y, z: x + y + z, range(N), range(N), range(N))
print('sync will block subsequent execution until computation is complete')
print(sum(res))
print('test')

Wall time: 1.64 s
sync will block subsequent execution until computation is complete
149999985000000
test


In [12]:
%time res = dview.map_async(lambda x, y, z: x + y + z, range(N), range(N), range(N))
print('async will allow subsequent execution')
print(sum(res))
print('until it hits a job that requires the result from the async job')

Wall time: 132 ms
async will allow subsequent execution
149999985000000
until it hits a job that requires the result from the async job


In [13]:
from ipyparallel import require

# sync all executions
dview.block = True 

# The "@remote" decorator creates functions which will execute simultaneously on every engine in a view.
@dview.remote()

# The "require" decorator forces the remote engine to import the packages we specify.
@require('numpy')

def f(n):
    return numpy.random.rand(n)

f(4)

[array([0.64092159, 0.75206439, 0.74750588, 0.28503196]),
 array([0.26599136, 0.78577109, 0.665103  , 0.40733447]),
 array([0.16512039, 0.64637858, 0.0060733 , 0.99312404]),
 array([0.99537792, 0.91740923, 0.56202473, 0.91835319]),
 array([0.50852651, 0.6661215 , 0.46786153, 0.27448539]),
 array([0.84170457, 0.40154229, 0.07270997, 0.92003936]),
 array([0.05809467, 0.81189237, 0.46823814, 0.59546814]),
 array([0.98103628, 0.42860423, 0.11474032, 0.90191469]),
 array([0.83785597, 0.55404756, 0.71402814, 0.50664071]),
 array([0.80222832, 0.55873752, 0.51555767, 0.91675295])]

In [15]:
# using sync_imports to import numpy on all engines
with dview.sync_imports():
    import numpy 

# px is a magic command that executes a single command to all engines
%px np = numpy # renaming

def f(n):
    n = n
    return np.random.rand(n)

dview.apply_sync(f,4)

importing numpy on engine(s)


[array([0.39084664, 0.35431275, 0.9917086 , 0.9239779 ]),
 array([0.15152262, 0.01598993, 0.07119199, 0.42848098]),
 array([0.94351683, 0.82293742, 0.41531075, 0.43453284]),
 array([0.94924306, 0.35638428, 0.55955577, 0.9261667 ]),
 array([0.64074127, 0.30005341, 0.83994591, 0.43484244]),
 array([0.30606774, 0.08005297, 0.35024904, 0.01267733]),
 array([0.28146231, 0.51602499, 0.75748782, 0.06421561]),
 array([0.20340654, 0.37026221, 0.07219586, 0.43831032]),
 array([0.41282971, 0.05039569, 0.34784081, 0.41015774]),
 array([0.78779674, 0.310394  , 0.01116404, 0.36760427])]

In [1]:
%reset -f

import ipyparallel as ipp
import multiprocessing as mp
import numpy as np
import math

rc = ipp.Client()
dview = rc[:]

In [2]:
S0 = 120.; K = 100.; B = 120.; T = 1.5; r = 0.02; σ = 0.3 # parameters
n_engs = len(rc) # number of engines running

In [3]:
# function that takes the parameters for pricing a option and the number of iterations required in the simulation
# returns the sum of the simulated option prices and the variance of the sample

def MC_BS_EKO_Call_v1(S0,K,B,T,σ,r,it,display=False):
    import math, numpy as np                          # another way to ensure modules are imported
    Z = math.sqrt(T)*np.random.standard_normal(it)    # generates normal random variables for the GBM
    S = S0*np.exp((r-0.5*σ**2)*T + σ*Z)               # simulates an array of final prices of asset at maturity of option
    KO = S<B                                          # generates an array of True=1 and False=0 based on barrier condition
    V = math.exp(-r*T)*np.maximum(S-K,0)*KO           # generates the array of option payoff based on final asset price
    return V

In [4]:
%%time

# Entire simulation on a single engine 

N = 10000000

V = MC_BS_EKO_Call_v1(S0,K,B,T,σ,r,N,True) 
price = np.average(V)
stderr = np.std(V, ddof=1) / N**0.5

# ddof = Delta Degrees of Freedom 
# the divisor used in calculations is N - ddof, where N represents the number of elements
# by default ddof is zero.

print('Single engine: total iterations =', N, 'Px={:.4}'.format(price), 'Std err = {:.4}'.format(stderr))

Single engine: total iterations = 10000000 Px=1.847 Std err = 0.001422
Wall time: 754 ms


In [5]:
%%time

# The same simulation split across the available engines:
# Since we are blocking until every result has come back, these timings
# will always reflect the performance of the slowest engine.

N_per_eng = int(N / n_engs) # number of iterations per engine

%time result = dview.apply_sync(MC_BS_EKO_Call_v1,S0,K,B,T,σ,r,N_per_eng) # returns all the results in an array of arrays
%time price = np.average(result) 
%time stderr = np.std(result, ddof=1)/N**0.5

print(n_engs, 'engines: total iterations =',N_per_eng*n_engs,'Px={:.4}'.format(price),'Std err = {:.4}'.format(stderr))

Wall time: 7.51 s
Wall time: 50.9 ms
Wall time: 106 ms
10 engines: total iterations = 10000000 Px=1.846 Std err = 0.001422
Wall time: 7.67 s


In [6]:
# Although it is simpler to only return all the arrays from each engine
# and calculate the price using average and variance on the combined set,
# it is a costly operation to 1) return the large arrays and 2) compute the 
# average and variance combined set

# let's redefine the function to return just 2 numbers: 
# average and variance of the sample for each engine

def MC_BS_EKO_Call_v2(S0,K,B,T,σ,r,it:int=1000000):
    import math, numpy as np
    Z = math.sqrt(T)*np.random.standard_normal(it)
    S = S0*np.exp((r-0.5*σ**2)*T + σ*Z)
    KO = S<B
    V = math.exp(-r*T)*np.maximum(S-K,0)*KO
    return (np.average(V), np.var(V))

In [15]:
%%time

# Recoding entire simulation on a single engine based on function returning average and variance
N=1000000

(V, var) = MC_BS_EKO_Call_v2(S0,K,B,T,σ,r,N_per_eng*n_engs) 
price = V
stderr = math.sqrt(var / (N-1))
# note np.var = sum(x_i-x_bar)^2 / N and stderr = sqrt[(sum(x_i-x_bar)^2 / N-1) / N] = sqrt[np.var/N-1]

print('Single engine: total iterations =', N, 'Px={:.4}'.format(price), 'Std err = {:.4}'.format(stderr))

Single engine: total iterations = 1000000 Px=1.846 Std err = 0.004494
Wall time: 118 ms


In [9]:
%%time

# Recoding simulation across multiple engines based on function returning average and variance

%time result = dview.apply_sync(MC_BS_EKO_Call_v2,S0,K,B,T,σ,r,N_per_eng)
print(result)

px = [x[0] for x in result]   # extracts all the averages of each engine's sample
print(px)
price = np.average(px)

# calculation of the standard error will be more complicated as we do not have the full sample
# it will require a piecewise calculation using the individual sample's mean and variance
# to get the total combined sample variance and hence standard error
# refer to the lecture slides on the formula to calculate the combined sample variance

φ = sum(var[1] for var in result)*N_per_eng                         # sum of numerators of variance for each engine's sample
for i in range(1, n_engs):                      
    φ += N_per_eng*i/(i+1)*(px[i] - np.average(px[:i]))**2          # loop to add the incremental adjustment factors 

stderr = math.sqrt((φ / (N-1)) / N) # calculate standard error using φ

print(n_engs, 'engines: total iterations =',N_per_eng*n_engs,'Px={:.4}'.format(price),'Std err = {:.4}'.format(stderr))

Wall time: 215 ms
[(1.8414765210124768, 20.170414971733138), (1.8470093569664014, 20.19142864593645), (1.841066363050665, 20.137012887842776), (1.842063310288634, 20.15703869834173), (1.849635826044783, 20.256095809844727), (1.8489134038191557, 20.235744878779293), (1.8512317013429915, 20.221690623910042), (1.8492052532887773, 20.25211793301807), (1.8473822538295228, 20.236816941268927), (1.84741809613775, 20.20089106215116)]
[1.8414765210124768, 1.8470093569664014, 1.841066363050665, 1.842063310288634, 1.849635826044783, 1.8489134038191557, 1.8512317013429915, 1.8492052532887773, 1.8473822538295228, 1.84741809613775]
10 engines: total iterations = 10000000 Px=1.847 Std err = 0.001421
Wall time: 216 ms


In [28]:
N = 5; steps = 4                       # 5 simulated paths each with 4 intermediate steps
prng = np.random.RandomState()   
Z = prng.standard_normal((N, steps))   # generate all standard normal random numbers required for each step in each path
print(Z)

[[-1.09920212  0.02814617 -0.60049384  1.2119334 ]
 [-0.23518205  0.38094641 -0.24474343 -0.99516516]
 [ 0.20904668 -0.25698729  0.10653497 -0.76020456]
 [ 0.24843272 -0.38548957  1.00368857  1.80314349]
 [ 0.45552077  0.15213503  0.18065519  0.75974051]]


In [112]:
T = 1; S0 = 100; β = 0.75              # parameters where starting asset price is 100
Δt = T/steps                           # size of each time step
S = S0*np.ones((N, steps + 1))         # generate matrix of size (N by steps) filled with starting asset price of 100

# loop to simulate all paths step by step using the numbers generated above
# note each step forward requires the asset price in the current time step
for t in range(steps):                 
    S[:,t+1] = S[:,t] + r*S[:,t]*Δt + σ*S[:,t]**β*math.sqrt(Δt)*Z[:,t] 
print(S)

[[100.          99.97691157  96.69955234  94.70472295  93.04609014]
 [100.         101.61224856 106.266483   109.11575461 109.5830422 ]
 [100.         109.97238059 101.83498576 101.0823923  108.16824512]
 [100.          97.63821779  99.75920614 103.62360852  95.43357783]
 [100.          95.29045732 102.07399861  93.19860111 104.82527704]]


In [10]:
# define a function to reuse the code for calculating the combined sample average and standard error

def process_result(result, n_engs, N_per_eng):    
    px = [x[0] for x in result]   
    price = np.average(px)
    
    φ = sum(var[1] for var in result)*N_per_eng                   # sum of numerators of variance for each engine's sample
    for i in range(1, n_engs):                      
        φ += N_per_eng*i/(i+1)*(px[i] - np.average(px[0:i]))**2   # loop to add the incremental adjustment factors 

    stderr = math.sqrt((φ / (n_engs*N_per_eng-1)) / (n_engs*N_per_eng))  # calculate standard error using φ

    return price, stderr

In [12]:
# For this exericise, we need to generate all the pseudo-random numbers at once 
# and then divide them among the engines to obtain meaningful timing and accuracy comparisons
# as both the single engine and multiple engine will be running on the same set of random numbers

def MC_CEV_Euler_EKO_Call(S0,K,B,T,σ,r,β,it,steps):
    import math, numpy as np
    Δt = T/steps
    S = S0*np.ones(it)                            # array of the initial asset price which will be updated at each step
    for t in range(steps):                      
        S += r*S*Δt + σ*S**β*math.sqrt(Δt)*z[:,t] # updating the asset price to t+1 using Euler scheme
                                                  # z is accessed as a global variable
    KO = S<B
    V = math.exp(-r*T)*np.maximum(S-K,0)*KO
    return (np.average(V), np.var(V))

def MC_CEV_Milstein_EKO_Call(S0,K,B,T,σ,r,β,it,steps):
    import math, numpy as np
    Δt = T/steps
    S = S0*np.ones(it)
    for t in range(steps):
        S += (r*S*Δt + σ*S**β*math.sqrt(Δt)*z[:,t] # updating the asset price to t+1 using Milstein scheme
              + 0.5*β*σ**2*S**(2*β-1)*Δt*(np.power(z[:,t],2)-1))
    KO = S<B
    V = math.exp(-r*T)*np.maximum(S-K,0)*KO
    return (np.average(V), np.var(V))

N=1000000; N_per_eng = int(N / n_engs)
β = 0.75; S0 = 120.; K = 100.; B = 120.; T = 1.5; r = 0.02; σ = 0.3

for steps in [5, 10, 20]:
    print('Using', steps, 'steps')
    
    prng = np.random.RandomState(1) 
    Z = prng.standard_normal((N_per_eng*n_engs,steps))
#     Z1 = prng.standard_normal((N_per_eng*n_engs,steps))
#     Z2 = prng.standard_normal((N_per_eng*n_engs,steps))
#     Z3 = prng.standard_normal((N_per_eng*n_engs,steps))
    
    # scatter function distributes the varible equally among the engines
    # which can be accessed as a global variable defined in the first argument
    dview.scatter('z', Z)
    
    result = dview.apply_sync(MC_CEV_Euler_EKO_Call,S0,K,B,T,σ,r,β,N_per_eng,steps)
#     (sum, var) = rc[0].apply_sync(MC_CEV_Euler_EKO_Call,S0,K,B,T,σ,r,β,N,steps,Z1)
    price, stderr = process_result(result, n_engs, N_per_eng)
    print('Euler Scheme: Price = {:.4}'.format(price), 'standard error = {:.4}'.format(stderr))
    
    result = dview.apply_sync(MC_CEV_Milstein_EKO_Call,S0,K,B,T,σ,r,β,N_per_eng,steps)
    price, stderr = process_result(result, n_engs, N_per_eng)
    print('Milstein Scheme: Price = {:.4}'.format(price), 'standard error = {:.4}'.format(stderr))

Using 5 steps
Euler Scheme: Price = 4.401 standard error = 0.006522
Milstein Scheme: Price = 4.457 standard error = 0.006541
Using 10 steps
Euler Scheme: Price = 4.422 standard error = 0.006526
Milstein Scheme: Price = 4.451 standard error = 0.006537
Using 20 steps
Euler Scheme: Price = 4.425 standard error = 0.006528
Milstein Scheme: Price = 4.439 standard error = 0.006533


In [13]:
%%time

import scipy.stats as sct

# Example of using antithetic variables to price the EKO call with Milstein scheme
# on the CEV model with beta = 0.75

N = 1000000; N_per_eng = int(N/n_engs); steps = 20

# create antithetic variables that will be scattered evenly across the engines
Z1 = sct.norm.ppf(np.random.uniform(0,1,(N//2,steps)))
Z2 = -Z1

# U1 = np.random.uniform(0,1,((N*n_engs)//2,steps))
# Z1 = sct.norm.ppf(U1)
# Z2 = sct.norm.ppf(1 - U1)

Z = np.append(Z1, Z2, axis=0)
dview.scatter('z', Z) 

result = dview.apply_sync(MC_CEV_Milstein_EKO_Call,S0,K,B,T,σ,r,β,N_per_eng,steps)
price, stderr = process_result(result, n_engs, N_per_eng)
print('Milstein Scheme: Price = {:.4}'.format(price), 'standard error = {:.4}'.format(stderr))

Milstein Scheme: Price = 4.442 standard error = 0.00653
Wall time: 3.25 s


In [91]:
%%time
import numpy as np
import math

# Using perfectly negatively correlated normal random variables as a mirroring set would
# create only almost perfectly negatively correlated underlying prices 
# However, the non-linear payoff function would cause the correlation coefficient to become
# less negatively correlated

# How about using the final underlying price created using the first set of standard normal
# to create a second set of underlying prices that is perfectly negatively correlated with
# the first set? Let's take a look

N=1000000; β = 0.75; S0 = 120.; K = 100.; B = 120.; T = 1.5; r = 0.02; σ = 0.3; steps = 20

Z1 = np.random.standard_normal((N//2,steps))
Z2 = -Z1 
Δt = T/steps
S1 = S0*np.ones(N//2)
S2 = S0*np.ones(N//2)

# Use the perfectly negatively correlation standard normal R.V. to generate second set 
# of price paths under the variable name S2

for t in range(steps):
    S1 += (r*S1*Δt + σ*S1**β*math.sqrt(Δt)*Z1[:,t] + 0.5*β*σ**2*S1**(2*β-1)*Δt*(np.power(Z1[:,t],2)-1))
    S2 += (r*S2*Δt + σ*S2**β*math.sqrt(Δt)*Z2[:,t] + 0.5*β*σ**2*S2**(2*β-1)*Δt*(np.power(Z2[:,t],2)-1))

print('Correlation of S1 and S2 = ', np.corrcoef(S1,S2)[0,1])

# calculate option prices by using the combined set of S1 and S2

KO = S1<B
V1 = math.exp(-r*T)*np.maximum(S1-K,0)*KO
KO = S2<B
V2 = math.exp(-r*T)*np.maximum(S2-K,0)*KO

V = np.append(V1,V2)
px = np.average(V)
stderr = np.sqrt(np.var(V)/(len(V)))

cov = np.cov(V1,V2, ddof=1)
rho = cov[0,1]/(cov[0,0]*cov[1,1])

print('Correlation of V1 and V2 = ', rho)
print('Px:', px)
print('Std Err:', stderr, '\n')

# Create a new set of final underlying price at maturity by reflecting the price
# about the initial price S0 and store it in variable S3 
# E.g. 130 reflects to 110 as S0 = 120


S3 = 2*S0 - S1
print('Correlation of S1 and S3 = ', np.corrcoef(S1,S3)[0,1])

# calculate option prices by using the combined set of S1 and S3

KO = S3<B
V3 = math.exp(-r*T)*np.maximum(S3-K,0)*KO

V = np.append(V1,V3)
px = np.average(V)
stderr = (np.var(V)/(len(V)))**0.5

cov = np.cov(V1,V3, ddof=1)
rho = cov[0,1]/(cov[0,0]*cov[1,1])

print('Correlation of V1 and V3 = ', rho)
print('Px:', px)
print('Std Err:', stderr)

# The price does not converge as the underlying prices in the 
# combined set of S1, S3 no longer follows the CEV distribution
# due to the way S3 is constructed

Correlation of S1 and S2 =  -0.9931722553117591
Correlation of V1 and V2 =  -0.010853763970465835
Px: 4.440156468690425
Std Err: 0.006528355944803806 

Correlation of S1 and S3 =  -1.0
Correlation of V1 and V3 =  -0.012411533775342291
Px: 4.795812386115853
Std Err: 0.006561752895497572
Wall time: 2.93 s


In [14]:
# Example of using S(T) as the control variable without using parallel processing

N = 1000000
convar = np.random.RandomState()
z = convar.standard_normal(N)
S = S0*np.exp((r - 0.5*σ**2)*T + z*(σ*math.sqrt(T)))

# calculate the average of S(T) which will be used as the control variable
avgSpot = np.average(S)

KO = S<B
V = math.exp(-r*T)*np.maximum(S-K,0)*KO

# generates the covariance matrix between option value and the control variable
# the adjustment factor for the error = -cov(S,V)/var(S)
# cov(S,V) = (var(S),cov(S,V); cov(S,V),var(V))
cov = np.cov(S, V, ddof=1) 
adj = -cov[0,1]/cov[0,0]    
print(cov)

price = np.sum(V)/N + adj*(avgSpot - S0*math.exp(r*T))
stderr = math.sqrt((cov[1,1] - cov[0,1]**2/cov[0,0])/N);
print('Control Variable: Price = {:.4}'.format(price), 'standard error = {:.4}'.format(stderr))
print('Correlation coefficient = {:.3}'.format(cov[0,1]/(math.sqrt(cov[0,0]*cov[1,1]))))

[[2203.83629089  -19.16951853]
 [ -19.16951853   20.15723816]]
Control Variable: Price = 1.841 standard error = 0.004471
Correlation coefficient = -0.091


In [13]:
from scipy.stats import norm

N = 1000000

# find the 2 points in the standard normal distribution 
# that correspond to S(T) = K and B
M = (math.log(B/S0)-(r-0.5*σ**2)*T)/(σ*math.sqrt(T)) # if Z < M then S(T) < B
L = (math.log(K/S0)-(r-0.5*σ**2)*T)/(σ*math.sqrt(T)) # if Z > L then S(T) > K

# generate uniform random numbers in (cdf(L),cdf(M)) for use with 
# the inverse CDF to generate required random numbers
impsam = np.random.RandomState() 
U = impsam.uniform(norm.cdf(L),norm.cdf(M),N)

# U provides probabilities in the interval that guarantees L < Z < M
# when put through the inverse CDF for standard normal distribution
# x are random variables that guarantee K < S(T) < B when used in
# the exact solution for S(T) using the GBM formula
x = norm.ppf(U)
S = S0*np.exp((r - 0.5*σ**2)*T + σ*math.sqrt(T)*x) 

# calculate the probabibility of K < S(T) < B 
ProbLtoM = norm.cdf(M) - norm.cdf(L)

# multiply the option payout by p(K<S(T)<B) to weigh outcome appropriately
# since we have generated a distribution that only produces outcomes in 
# the range of K<S(T)<B
V = math.exp(-r*T)*np.maximum(0, S - K)* ProbLtoM

price = np.sum(V) / N 
stderr = np.std(V,ddof=1)/math.sqrt(N)

print('Importance Sampling: Price = {:.4}'.format(price), 'standard error = {:.4}'.format(stderr))

Importance Sampling: Price = 1.846 standard error = 0.001082


In [94]:
import scipy.stats as sct

N = 1000000

def StratSampling(S0,K,B,T,σ,r,N,Nb):
    prng = np.random.RandomState(0)
    bins = np.linspace(0, 1, Nb+1)
    U = []
    for b in range(Nb):
        # create bins of uniform RVs in respective interval of each bin
        U.append(prng.uniform(bins[b], bins[b+1], int(N/Nb))) 
    
    # use the standard normal inverse CDF on the uniform RVs 
    # to generate corresponding standard normal RVs in each bin
    Z = sct.norm.ppf(U)
        
    # standard MC procedure
    S = S0*np.exp((r - 0.5*σ**2)*T + Z*(σ*math.sqrt(T)))
    KO = S<B
    V = math.exp(-r*T)*np.maximum(S-K,0)*KO
    
    price = np.sum(V)/N
    
    # variance calculated bin by bin
    var = np.var(V,ddof=1,axis=1)
    
    # variance combined and standard error of estimate is calculated
    stderr = math.sqrt(np.sum(var)/(Nb*N))
    
    return (price, stderr)

for Nb in [10, 100, 1000, 10000, 100000, 500000, 1000000]:
    (price, stderr) = StratSampling(S0,K,B,T,σ,r,N,Nb)
    print(Nb, 'bins: Price = {:.4}'.format(price), 'standard error = {:.4}'.format(stderr))

10 bins: Price = 1.846 standard error = 0.002888
100 bins: Price = 1.847 standard error = 0.000498
1000 bins: Price = 1.847 standard error = 0.0002922
10000 bins: Price = 1.847 standard error = 9.735e-05
100000 bins: Price = 1.847 standard error = 2.965e-05
500000 bins: Price = 1.847 standard error = 2.544e-08
1000000 bins: Price = 1.847 standard error = nan


C:\Users\luchu\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3584: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\luchu\Anaconda3\lib\site-packages\numpy\core\_methods.py:207: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
